# IR final Project

## Setup

In [ ]:
!pip install -q graphframes
!pip install google-cloud-storage==3.7.0
!pip install --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install sentence-transformers
import google.cloud.storage
print(google.cloud.storage.__version__)
# !pip install --upgrade google-cloud-storage --ignore-installed

  Attempting uninstall: proto-plus
    Found existing installation: proto-plus 1.11.0
    Uninstalling proto-plus-1.11.0:
      Successfully uninstalled proto-plus-1.11.0
  Attempting uninstall: google-resumable-media
    Found existing installation: google-resumable-media 1.3.3
    Uninstalling google-resumable-media-1.3.3:
      Successfully uninstalled google-resumable-media-1.3.3
  Attempting uninstall: google-auth
    Found existing installation: google-auth 1.35.0
    Uninstalling google-auth-1.35.0:
      Successfully uninstalled google-auth-1.35.0
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.34.1
    Uninstalling google-api-core-1.34.1:
      Successfully uninstalled google-api-core-1.34.1
  Attempting uninstall: google-cloud-core
    Found existing installation: google-cloud-core 1.7.3
    Uninstalling google-cloud-core-1.7.3:
      Successfully uninstalled google-cloud-core-1.7.3
  Attempting uninstall: google-cloud-storage
    Fo

In [ ]:
import nltk
import numpy as np
import bz2
import codecs
import csv
import time
import os
import re
from pathlib import Path
from typing import List, Tuple
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
from itertools import islice, count, groupby
import pandas as pd
from operator import itemgetter
from nltk.stem.porter import *
from nltk.corpus import stopwords
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *
import pickle
from google.cloud import storage
import math
from sentence_transformers import SentenceTransformer, util
from nltk.stem import PorterStemmer
import pyspark.sql.functions as F 
from pyspark.sql.types import IntegerType, StructType, StructField
from pyspark.sql.functions import sum as _sum, count as _count
import os
import pickle
from pathlib import Path
from collections import Counter


%cd -q /home/dataproc
!ls inverted_index_gcp.py
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())
from inverted_index_gcp import InvertedIndex

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

/opt/conda/miniconda3/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


inverted_index_gcp.py


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Consts

In [ ]:
# consts
project_id = 'ir-final-project-12016'
data_bucket_name = 'ir-final-project-bucket'
output_folder = "mappings"
storage_output_folder = "prod"
page_view_map_filename = "pageviews-202108-user.pkl"
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
prod_file_name = f"gs://{data_bucket_name}/multistream*"
final_index_filename = "pageview_dict.pkl" 

## Download the data

In [ ]:
# Download page views
!wget $pv_path
# download data wikidumps
!mkdir -p wikidumps
!gsutil -u {project_id} cp gs://{data_bucket_name}/multistream*_preprocessed.parquet "wikidumps/"

# load the data
pages_links = spark.read.parquet(prod_file_name).select("anchor_text", "id").rdd
pages_title = spark.read.parquet(prod_file_name).select("title", "id").rdd
pages_body = spark.read.parquet(prod_file_name).select("text", "id").rdd

Copying gs://ir-final-project-bucket/multistream10_preprocessed.parquet...
Copying gs://ir-final-project-bucket/multistream11_part2_preprocessed.parquet...
Copying gs://ir-final-project-bucket/multistream11_preprocessed.parquet...      
Copying gs://ir-final-project-bucket/multistream12_part2_preprocessed.parquet...
/ [4 files][  1.2 GiB/  1.2 GiB]   76.2 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://ir-final-project-bucket/multistream12_preprocessed.parquet...
Copying gs://ir-final-project-bucket/multistream13_part2_preprocessed.parquet...
Copying gs://ir-final-project-bucket/multistream13_preprocessed.parquet...      
Copying gs://ir-final-project-bucket/multistream14_part2_preprocessed.parquet...
Copying gs://ir-final-project-b

## Page View Task Solution

### Data Creation

In [ ]:

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def read_page_view_and_create_map():
    p = Path(pv_path)
    pv_name = p.name  # pageviews-202108-user.bz2
    pv_temp = f'{p.stem}-4dedup.txt'
        
    # the data wasnt downloaded
    if not os.path.exists(pv_name):
        print(f"Error: File {pv_name} not found - download the data using wget.")
        return None
    # Filter for English pages, and keep just two fields: article ID (3) and monthly
    # total number of page views (5). Then, remove lines with article id or page
    # view values that are not a sequence of digits.
    command = f'bzcat {pv_name} | grep "^en\.wikipedia" | cut -d" " -f3,5 | grep -P "^\d+\s\d+$" > {pv_temp}'
    exit_code = os.system(command)
    
    if exit_code != 0:
        print("Error in shell command")
        return None

    print("Building dictionary in Python...")
    # Create a Counter (dictionary) that sums up the pages views for the same
    # article, resulting in a mapping from article id to total page views.
    wid2pv = Counter()
    
    try:
        with open(pv_temp, 'rt') as f:
            for line in f:
                parts = line.strip().split(' ')
                wid2pv.update({int(parts[0]): int(parts[1])})
                
        pv_dict = dict(wid2pv)
        
        destination = f"{output_folder}/{final_index_filename}"
        with open(destination, "wb") as f:
            pickle.dump(pv_dict, f)
                
        if os.path.exists(pv_temp):
            os.remove(pv_temp)
            
        return pv_dict

    except Exception as e:
        print(f"Error during python processing: {e}")
        return None

print("Starting execution...")
dictionary_result = read_page_view_and_create_map()

Starting execution...
Processing file: pageviews-202108-user.bz2...
Building dictionary in Python...
Success! Saved to mappings/pageview_dict.pkl
Uploading mappings/pageview_dict.pkl to gs://ir-final-project-bucket/prod/mappings/pageview_dict.pkl...
Copying file://mappings/pageview_dict.pkl [Content-Type=application/octet-stream]...
- [1 files][ 73.5 MiB/ 73.5 MiB]                                                
Operation completed over 1 objects/73.5 MiB.                                     

--- Test Results ---
[335, 11706, 925]


In [ ]:
# upload ouput page view to storage
index_src = f"{output_folder}/{final_index_filename}"
index_dst = f'gs://ir-final-project-bucket/{storage_output_folder}/mappings/{final_index_filename}'
print(f"Uploading {index_src} to {index_dst}...")
!gsutil cp $index_src $index_dst

### Prediction

In [ ]:
# load
with open(f"{output_folder}/pageview_dict.pkl", 'rb') as f:
    wid2pv = pickle.loads(f.read())


def get_pageview(wiki_ids: List[int]) -> List[int]:
  return [wid2pv[wiki_id] for wiki_id in wiki_ids]

#check func
get_pageview([27960185, 1232201, 1836501])

## Page Rank Solution

### Data Creation

In [ ]:
def extract_edges_from_row(row):
  #row: (source_id, list_of_anchors)
  #list_of_anchors: [(destination_id, link_text),...]
    source_id = row[1]
    list_of_anchors = row[0]
    edges = []
    # iterate through the anchors list to extract destination IDs
    if list_of_anchors is not None:
        for anchor in list_of_anchors:
            destination_id = anchor[0]
            edges.append((source_id, destination_id))

    return edges

def get_first_element(edge):
    return edge[0]

def get_second_element(edge):
    return edge[1]

def generate_graph(pages):
    ''' Compute the directed graph generated by wiki links.
    Parameters:
    -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
    Returns:
    --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
    '''
    # compute edges: transform each page into a list of individual edges -> (source_id, destination_id) pairs
    edges_with_duplicates = pages.flatMap(extract_edges_from_row)

    # remove duplicate edges
    edges = edges_with_duplicates.distinct()

    # compute Vertices: extract all source IDs from the edges
    sources = edges.map(get_first_element)

    # extract all destination IDs from the edges
    destinations = edges.map(get_second_element)

    # union both lists and removes duplicates
    vertices = sources.union(destinations).distinct().map(lambda x: (x,))

    return edges, vertices


def create_page_rank(pages_df):
    # construct the graph
    edges, vertices = generate_graph(pages_df)
    # compute PageRank
    edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
    verticesDF = vertices.toDF(['id']).repartition(124, 'id')
    g = GraphFrame(verticesDF, edgesDF)
    pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
    pr = pr_results.vertices.select("id", "pagerank")
    pr = pr.sort(col('pagerank').desc())
    pr_pd = pr.toPandas()
#     pr.write.mode("overwrite").parquet(pr_path)
    pr_dict = dict(zip(pr_pd['id'].astype(int), pr_pd['pagerank']))
    with open(f"{output_folder}/pagerank_dict.pkl", "wb") as f:
        pickle.dump(pr_dict, f)
    storage.Client().bucket(data_bucket_name).blob(f"{storage_output_folder}/mappings/pagerank_dict.pkl").upload_from_filename(
        f"{output_folder}/pagerank_dict.pkl"
    )

create_page_rank(pages_links)

### Prediction

In [ ]:
#load
storage.Client().bucket(data_bucket_name).blob(f"{storage_output_folder}/mappings/pagerank_dict.pkl").download_to_filename("pagerank_dict.pkl")
with open("pagerank_dict.pkl", "rb") as f:
    PAGERANK_DICT = pickle.load(f)
def get_pagerank(wiki_ids: List[int]) -> List[float]:
    res = [PAGERANK_DICT.get(int(wid), 0.0) for wid in wiki_ids]
    return res

#check
get_pagerank([21241, 21148, 9316])

[0.0, 1959.6382627292862, 2782.03964641377]

## Inverted Index Creation - Data

In [ ]:
client = storage.Client()

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

if not os.path.exists(data_bucket_name):
    os.makedirs(data_bucket_name)

# toknization for the basic inverted indecies
def simple_tokenize(full_text: str) -> List[str]:
    return  [
        tok.group().lower()
        for tok in RE_WORD.finditer(full_text)
        if tok.group().lower() not in all_stopwords
    ]

#warpper function for the toknization function
def tokenize_text(text_element, tokenize_func):
    if isinstance(text_element, list):
        if not text_element:
            return []
        texts = []
        for r in text_element:
            if hasattr(r, "text") and isinstance(r.text, str):
                texts.append(r.text)
        if not texts:
            return []
        full_text = " ".join(texts)
    else:
        full_text = text_element

    return tokenize_func(full_text)

def word_count(text_element, doc_id, tokenize_func):
    tf_counter = Counter(tokenize_text(text_element, tokenize_func))
    return [(token, (doc_id, tf)) for token, tf in tf_counter.items()]

def reduce_word_counts(text_element):
    return sorted(text_element, key=lambda x: x[0])

def calculate_df(postings):
    return postings.mapValues(lambda pl: len(pl))

def map_to_bucket_key(item):
    """ Transforms (token, posting_list) into (bucket_id, (token, posting_list)). """

    # item[0] is the token
    bucket_id = token2bucket_id(item[0])

    # the value is the original item itself
    return (bucket_id, item)

def write_and_return_locations(bucket_id_and_postings, index_name: str):
    """ Writes one bucket's postings to disk and returns the location map. """
    os.makedirs(data_bucket_name, exist_ok=True)
    # unpack the input: (bucket_id, iterable_of_postings)
    bucket_id, postings_iterable = bucket_id_and_postings

    # [(w, posting_list), (w, posting_list)....]
    bucket_postings_list = list(postings_iterable)
    if not bucket_postings_list:
        return []

    posting_locations_dict = InvertedIndex.write_a_posting_list((bucket_id, bucket_postings_list), f"{storage_output_folder}/{index_name}_gcp", data_bucket_name)
    return posting_locations_dict

def partition_postings_and_write(postings, index_name: str):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
    '''
    # add the bucket_id as the key -> (bucket_id, (w, posting_list))
    mapped_rdd = postings.map(map_to_bucket_key)

    # collect all items belonging to the same bucket_id -> (bucket_id, iterator of (w, posting_list))
    grouped_rdd = mapped_rdd.groupByKey()

    # apply the disk writing function to each grouped bucket
    # return information and the location of the data in the disk
    return grouped_rdd.map(lambda x: write_and_return_locations(x, index_name))

def create_inverted_index(pages_data: pyspark.rdd.RDD, index_name: str, min_elements_in_doc: int, tokenize_func = simple_tokenize):
    word_counts = pages_data.flatMap(lambda x: word_count(x[0], x[1], tokenize_func))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    # filtering postings and calculate df
    postings_filtered = postings.filter(lambda x: len(x[1])>min_elements_in_doc)
    
    postings_filtered.persist()
    print(f"Writing index {index_name} to disk...")
    # partition posting lists and write out
    _ = partition_postings_and_write(postings_filtered, index_name).collect()
    
    print("Calculating and collecting DF...")
    w2df = calculate_df(postings_filtered)
    w2df_dict = w2df.collectAsMap()

    inverted = InvertedIndex()

    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(data_bucket_name, prefix=f"{storage_output_folder}/{index_name}_gcp"):
        if not blob.name.endswith("pickle"):
            continue
        with blob.open("rb") as f:
            posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

    inverted.posting_locs = super_posting_locs
    inverted.df = w2df_dict

    # write the global stats out
    inverted.write_index('.', index_name)
    index_src = f"{index_name}.pkl"
    index_dst = f'gs://{data_bucket_name}/{storage_output_folder}/inverted_indecies/{index_src}'
    !gsutil cp $index_src $index_dst

### all basic index creation

In [ ]:
create_inverted_index(pages_links, "ancher_inverted_index", 10)

In [ ]:
create_inverted_index(pages_title, "title_inverted_index", 0)

In [ ]:
create_inverted_index(pages_body, "body_inverted_index", 50)

Writing index body_inverted_index to disk...


Exception in thread "serve RDD 37" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)
25/12/25 12:42:33 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_42_35 !
25/12/25 12:42:33 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_42_123 !
25/12/25 12:42:33 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_42_1 !
25/12/25 12:42:33 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_42_96 !
25/12/25 12:42:33 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available

Calculating and collecting DF...


Copying file://body_inverted_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 18.5 MiB/ 18.5 MiB]                                                
Operation completed over 1 objects/18.5 MiB.                                     


### Basic Search predictions

In [ ]:
def load_resources():
    client = storage.Client()
    files = ["titles_dict.pkl", "anchor_stats.pkl", "title_stats.pkl", "body_stats.pkl"]
    for file_name in files:
        client.bucket(data_bucket_name).blob(f"{storage_output_folder}/stats/{file_name}.pkl").download_to_filename(file_name)
    with open('body_stats.pkl', 'rb') as f:
        body_data = pickle.load(f)
    with open('title_stats.pkl', 'rb') as f:
        title_data = pickle.load(f)
    with open('anchor_stats.pkl', 'rb') as f:
        anchor_data = pickle.load(f)
    with open('titles_dict.pkl', 'rb') as f:
        titles_dict = pickle.load(f)
    client.bucket(data_bucket_name).blob(f"{storage_output_folder}/mappings/pagerank_dict.pkl").download_to_filename("pagerank_dict.pkl")
    with open("pagerank_dict.pkl", "rb") as f:
        pagerank_dict = pickle.load(f)
    client.bucket(data_bucket_name).blob(f"{storage_output_folder}/mappings/pageview_dict.pkl").download_to_filename("pageview_dict.pkl")
    with open("pageview_dict.pkl", "rb") as f:
        wid2pv = pickle.load(f)

    return body_data, title_data, anchor_data, titles_dict, pagerank_dict, wid2pv


BODY_STATS, TITLE_STATS, ANCHOR_STATS, TITLES_DICT, PAGERANK_DICT, WID2PV = load_resources()


In [ ]:
# read the inverted index from storage
def read_inverted_index(index_name: str):
    # read the inverted index from storage
    index_src = f'gs://{data_bucket_name}/{storage_output_folder}/inverted_indecies/{index_name}.pkl'
    index_dst = f"{index_name}.pkl"
    !gsutil cp $index_src $index_dst
    with open(index_dst, "rb") as f:
        return pickle.load(f)

body_inverted_index = read_inverted_index("body_inverted_index")
title_inverted_index = read_inverted_index("title_inverted_index")
inverted_anchor_index = read_inverted_index("ancher_inverted_index")

def search_helper(query: str, inverted_index):
    query_tokens = {
        tok.group().lower()
        for tok in RE_WORD.finditer(query)
        if tok.group().lower() not in all_stopwords
    }
    if not query_tokens:
        return []

    # Map doc_id -> set of query tokens that appear in its anchor text
    doc_seen_per_token = defaultdict(set)

    # Only process posting lists for terms in the query
    for term in query_tokens:
        posting_list = inverted_index.read_a_posting_list('.', term, data_bucket_name)
        for doc_id, tf in posting_list:
            doc_seen_per_token[doc_id].add(term)

    # Count distinct query words per document
    doc2count = [(doc_id, len(token_set)) for doc_id, token_set in doc_seen_per_token.items()]

    # Sort by number of distinct query words descending
    doc2count.sort(key=lambda x: -x[1])
    if len(doc2count) == 0:
        return []
    res = []
    for doc_id, count in doc2count:
        title = TITLES_DICT.get(str(doc_id), "Unknown")
        res.append((str(doc_id), title))
    return res


Copying gs://final-project-bucket-ir/inverted_indecies_test/body_inverted_index.pkl...
/ [1 files][  7.6 MiB/  7.6 MiB]                                                
Operation completed over 1 objects/7.6 MiB.                                      
Copying gs://final-project-bucket-ir/inverted_indecies_test/title_inverted_index.pkl...
/ [1 files][433.2 KiB/433.2 KiB]                                                
Operation completed over 1 objects/433.2 KiB.                                    
Copying gs://final-project-bucket-ir/inverted_indecies_test/ancher_inverted_index.pkl...
/ [1 files][  3.1 MiB/  3.1 MiB]                                                
Operation completed over 1 objects/3.1 MiB.                                      


In [ ]:
def search_title(query: str) -> List[Tuple[str, str]]:
    return search_helper(query, title_inverted_index)

def search_anchor(query: str) -> List[Tuple[str, str]]:
    return search_helper(query, inverted_anchor_index)

def search_body(query: str) -> List[Tuple[str, str]]:
    return search_helper(query, body_inverted_index)

## Multi Search Data Creation

### Inverted Index with better tokenizer

In [ ]:
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')
MAX_BIGRAMS = 30     
MIN_TOKEN_LEN = 3

def tokenize_anchor(text):
    stemmer = PorterStemmer()
    return [
        # stemming
        stemmer.stem(tok.group().lower())
        for tok in RE_WORD.finditer(text)
        if tok.group().lower() not in all_stopwords
        and len(tok.group()) >= MIN_TOKEN_LEN
    ]

def tokenize_smart(text):
    # stemming
    stemmer = PorterStemmer()
    # basic tokenize 
    tokens = [tok.group().lower()for tok in RE_WORD.finditer(text)]
    tokens = [stemmer.stem(tok)for tok in tokens if tok not in all_stopwords and len(tok) >= MIN_TOKEN_LEN]
    # create bigrams
    bigrams = []
    for i in range(len(tokens) - 1):
        if len(bigrams) >= MAX_BIGRAMS:
            break
        t1, t2 = tokens[i], tokens[i + 1]
        if t1.isalpha() and t2.isalpha():
            bigrams.append(f"{t1}_{t2}")
    return tokens + bigrams


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
create_inverted_index(pages_links, "ancher_inverted_index_smart", 10, tokenize_func=tokenize_anchor)

Exception in thread "serve RDD 34" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


Copying file://ancher_inverted_index_smart.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 21.4 MiB/ 21.4 MiB]                                                
Operation completed over 1 objects/21.4 MiB.                                     


In [ ]:
create_inverted_index(pages_title, "title_inverted_index_smart", 0, tokenize_func=tokenize_smart)

Copying file://title_inverted_index_smart.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1 files][279.9 MiB/279.9 MiB]                                                
Operation completed over 1 objects/279.9 MiB.                                    


### avg text len map

In [ ]:
def count_tokens_fast(text_element):
    if text_element is None:
        return 0   
    # handeling anchor
    if isinstance(text_element, list):
        text_parts = []
        for item in text_element:
            if isinstance(item, str):
                text_parts.append(item)
            elif hasattr(item, "text") and item.text:
                text_parts.append(item.text)
            elif isinstance(item, (list, tuple)) and len(item) > 1: 
                 text_parts.append(str(item[1]))
        
        text_element = " ".join(text_parts)
    
    # another final check
    if not isinstance(text_element, str):
        return 0

    #extracting tokens and filtering stop words    
    tokens = [tok.group().lower() for tok in RE_WORD.finditer(text_element)]
    tokens = [tok for tok in tokens if tok not in all_stopwords and len(tok) >= MIN_TOKEN_LEN]

    bigrams_count = 0
    token_count = len(tokens)
    # creating bigrams
    for i in range(token_count - 1):
        if bigrams_count >= MAX_BIGRAMS:
            break
            
        if tokens[i].isalpha() and tokens[i+1].isalpha(): # bigram should be an actual word and not a number
            bigrams_count += 1
            
    return token_count + bigrams_count

# bm25 needs avg doc len so we calcluate it for body, title and anchor
def calculate_field_stats(rdd, field_name):
    print(f"Starting stats calculation for {field_name}...")
    parquet_path = f"dl_{field_name}"
    
    if field_name == 'body': # too much data for body so we go with the statstics
        print("Sampling 10% of the body data to speed up calculation...")
        rdd = rdd.sample(False, 0.1, seed=42)
    
    dl_rdd = rdd.map(lambda x: (str(x[1]), count_tokens_fast(x[0])))
    dl_rdd.toDF(["doc_id", "dl"]).write.mode("overwrite").parquet(parquet_path)
    
    df_stats = spark.read.parquet(parquet_path)
    agg_results = df_stats.select(
        _sum("dl").alias("total_len"), 
        _count("dl").alias("doc_count")
    ).collect()[0]
    
    total_len = agg_results["total_len"]
    sample_doc_count = agg_results["doc_count"]
    
    avg_len = total_len / sample_doc_count if sample_doc_count else 0
    real_doc_count = sample_doc_count * 10 if field_name == 'body' else sample_doc_count
    
    stats = {"avgdl": avg_len, "doc_count": real_doc_count}
    filename = f"{field_name}_stats.pkl"
    with open(filename, "wb") as f:
        pickle.dump(stats, f)
        
    print(f"Finished {field_name}: avgdl={avg_len}, est_count={real_doc_count}")
    return filename

In [ ]:
calculate_field_stats(pages_body, 'body')

Starting stats calculation for body...
Sampling 10% of the body data to speed up calculation...


Finished body: avgdl=346.72521575500065, est_count=6345160


'body_stats.pkl'

In [ ]:
calculate_field_stats(pages_title, 'title')

Starting stats calculation for title...


Finished title: avgdl=3.8594021650960557, est_count=6348910


'title_stats.pkl'

In [ ]:
calculate_field_stats(pages_links, 'anchor')

Starting stats calculation for anchor...


Finished anchor: avgdl=95.7779291248419, est_count=6348910


'anchor_stats.pkl'

In [ ]:
#upload avg len to storage
files = ["anchor_stats.pkl","title_stats.pkl", "body_stats.pkl"]
for file_name in files:
    index_dst = f'gs://{data_bucket_name}/{storage_output_folder}/stats/{file_name}.pkl'
    !gsutil cp $file_name $index_dst

### DOC_ID -> TITLE Mapping - uploading a parquet file for pandas

In [ ]:
# download mapping between doc_id to title for the search_frontend api to replace each doc id in result to a real title - will be read with pandas later
spark.read.parquet(prod_file_name).select("title", "id").write.mode("overwrite").parquet(f"gs://{data_bucket_name}/{storage_output_folder}/titles_df")

In [ ]:
# download df code for prediction in search frontend api
def download_df():
    prefix = f"{storage_output_folder}/titles_df"
    blobs = storage.Client().bucket(data_bucket_name).list_blobs(prefix=prefix)
    for blob in blobs:
        print(blob.name)
        if blob.name.endswith("/"): 
            continue # Skip directory placeholders
        rel_path = os.path.relpath(blob.name, start=os.path.dirname(prefix))
        local_file_path = os.path.join(".", rel_path)
        os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
        print(f"Downloading {rel_path}...")
        blob.download_to_filename(local_file_path)
download_df()

prod/titles_df/
prod/titles_df/_SUCCESS
prod/titles_df/part-00000-4050438b-a9de-4ca6-b56c-536e8d275a91-c000.snappy.parquet
prod/titles_df/part-00002-4050438b-a9de-4ca6-b56c-536e8d275a91-c000.snappy.parquet
prod/titles_df/part-00005-4050438b-a9de-4ca6-b56c-536e8d275a91-c000.snappy.parquet
prod/titles_df/part-00007-4050438b-a9de-4ca6-b56c-536e8d275a91-c000.snappy.parquet
prod/titles_df/part-00009-4050438b-a9de-4ca6-b56c-536e8d275a91-c000.snappy.parquet
prod/titles_df/part-00010-4050438b-a9de-4ca6-b56c-536e8d275a91-c000.snappy.parquet
prod/titles_df/part-00012-4050438b-a9de-4ca6-b56c-536e8d275a91-c000.snappy.parquet
prod/titles_df/part-00013-4050438b-a9de-4ca6-b56c-536e8d275a91-c000.snappy.parquet
prod/titles_df/part-00015-4050438b-a9de-4ca6-b56c-536e8d275a91-c000.snappy.parquet
prod/titles_df/part-00016-4050438b-a9de-4ca6-b56c-536e8d275a91-c000.snappy.parquet
prod/titles_df/part-00017-4050438b-a9de-4ca6-b56c-536e8d275a91-c000.snappy.parquet
prod/titles_df/part-00018-4050438b-a9de-4ca6-b5

In [ ]:
# index the dataframe and testing the mapping
df = pd.read_parquet("titles_df", engine='pyarrow')
df_indexed = df.set_index("id")["title"]
df_indexed.get(4045403, "Unknown Title")

'Foster Air Force Base'

### Data for BM25 - ID -> Doc Len

In [ ]:
def count_tokens_precise(text_element, field_type):
    """
    Counts tokens based on the specific tokenizer used for the field.
    field_type: 'body' (simple), 'anchor' (stemmed/simple), 'title' (smart/bigrams)
    """
    if text_element is None:
        return 0
    
    # pre proccesing
    full_text = ""
    if isinstance(text_element, (list, tuple)):
        # Handle list of Rows or Tuples (common in Anchor text)
        text_parts = []
        for item in text_element:
            if isinstance(item, str):
                text_parts.append(item)
            elif hasattr(item, "text") and item.text: # Spark Row
                text_parts.append(item.text)
            elif isinstance(item, (list, tuple)) and len(item) > 1: # Tuple
                text_parts.append(str(item[1]))
        full_text = " ".join(text_parts)
    elif isinstance(text_element, str):
        full_text = text_element
    else:
        return 0

    # Extract base tokens using Regex
    tokens = [tok.group().lower() for tok in RE_WORD.finditer(full_text)]
    # Filter Stopwords & Length
    tokens = [t for t in tokens if t not in all_stopwords and len(t) >= MIN_TOKEN_LEN]
    
    token_count = len(tokens)

    # Only 'title' used tokenize_smart which adds bigrams. 
    # 'body' and 'anchor' used simple/stem tokenizers without bigrams.
    if field_type == 'title':
        bigrams_count = 0
        for i in range(token_count - 1):
            if bigrams_count >= MAX_BIGRAMS:
                break
            if tokens[i].isalpha() and tokens[i+1].isalpha():
                bigrams_count += 1
        return token_count + bigrams_count
    
    # For Body and Anchor, return simple token count
    return token_count


def calculate_dl_and_save(rdd, field_name):
    """
    Calculates DL, writes directly to Parquet, and prints stats.
    """
    print(f"\n--- Processing {field_name} ---")
    
    # Define Mapper
    def mapper(row):
        # Extract ID and Text robustly
        doc_id = row['id'] if hasattr(row, 'id') else row[1]
        
        # Select content based on known schema columns
        if field_name == 'anchor' and hasattr(row, 'anchor_text'):
            content = row.anchor_text
        elif field_name == 'title' and hasattr(row, 'title'):
            content = row.title
        elif field_name == 'body' and hasattr(row, 'text'):
            content = row.text
        else:
            content = row[0] # Fallback for tuple (text, id)

        # Count tokens using the logic specific to the field type
        length = count_tokens_precise(content, field_type=field_name)
        return (int(doc_id), int(length))

    # Create DataFrame (id, len)
    dl_rdd = rdd.map(mapper)
    schema = StructType([
        StructField("id", IntegerType(), True),
        StructField("len", IntegerType(), True)
    ])
    df = spark.createDataFrame(dl_rdd, schema)

    # Define Output Path
    output_path = f"gs://{data_bucket_name}/{storage_output_folder}/stats/dl_{field_name}.parquet" # remove .parquet if not work
    
    # Write to Parquet (Action that triggers the job)
    print(f"Writing to: {output_path}")
    df.write.mode("overwrite").parquet(output_path)
    

In [11]:
calculate_dl_and_save(pages_title, 'title')


--- Processing title ---
Writing to: gs://ir-final-project-bucket/prod/stats/dl_title.parquet


In [12]:
calculate_dl_and_save(pages_links, 'anchor')


--- Processing anchor ---
Writing to: gs://ir-final-project-bucket/prod/stats/dl_anchor.parquet


In [ ]:
calculate_dl_and_save(pages_body, 'body')


--- Processing body ---
Writing to: gs://ir-final-project-bucket/prod/stats/dl_body.parquet


In [ ]:
# validate all the data is good
src = f"gs://{data_bucket_name}/{storage_output_folder}/stats/dl_body.parquet"
!gsutil -m cp -r $src .

df = pd.read_parquet("dl_body.parquet", engine='pyarrow')
df_indexed = df.set_index("id")["len"]
df_indexed.get(4045403, 0)

Copying gs://ir-final-project-bucket/prod/stats/dl_body.parquet/_SUCCESS...
Copying gs://ir-final-project-bucket/prod/stats/dl_body.parquet/part-00000-cf64c327-2a9c-47ac-ba94-ae9e16ad4cc4-c000.snappy.parquet...
Copying gs://ir-final-project-bucket/prod/stats/dl_body.parquet/part-00002-cf64c327-2a9c-47ac-ba94-ae9e16ad4cc4-c000.snappy.parquet...
Copying gs://ir-final-project-bucket/prod/stats/dl_body.parquet/part-00005-cf64c327-2a9c-47ac-ba94-ae9e16ad4cc4-c000.snappy.parquet...
Copying gs://ir-final-project-bucket/prod/stats/dl_body.parquet/part-00013-cf64c327-2a9c-47ac-ba94-ae9e16ad4cc4-c000.snappy.parquet...
Copying gs://ir-final-project-bucket/prod/stats/dl_body.parquet/part-00027-cf64c327-2a9c-47ac-ba94-ae9e16ad4cc4-c000.snappy.parquet...
Copying gs://ir-final-project-bucket/prod/stats/dl_body.parquet/part-00009-cf64c327-2a9c-47ac-ba94-ae9e16ad4cc4-c000.snappy.parquet...
Copying gs://ir-final-project-bucket/prod/stats/dl_body.parquet/part-00028-cf64c327-2a9c-47ac-ba94-ae9e16ad4cc4-c0

1517

### Search - Prediction

In [ ]:
body_inverted_index_smart = read_inverted_index("body_inverted_index")
title_inverted_index_smart = read_inverted_index("title_inverted_index_smart")
anchor_inverted_index_smart = read_inverted_index("ancher_inverted_index_smart")

In [ ]:
def get_bm25_scores(query: str):
    """
    Calculates BM25 scores for a given query across multiple index fields 
    (Body, Title, Anchor text).
    """
    # BM25 Hyperparameters:
    # k1: Controls term frequency saturation 
    # b: Controls document length normalization 
    k1, b = 1.5, 0.75
    # Dictionary to store cumulative scores for each document ID
    candidate_scores = defaultdict(float)

    # Configuration for Multi-Field Retrieval
    fields = [
        (body_inverted_index_smart, BODY_STATS, 0.5, simple_tokenize),
        (title_inverted_index_smart, TITLE_STATS, 0.3, tokenize_smart),
        (anchor_inverted_index_smart, ANCHOR_STATS, 0.2, tokenize_anchor)
    ]

    # Iterate over each field (Body, Title, Anchor) to calculate partial scores
    for index, stats, weight, tokenize_func in fields:
        # Tokenize the query using the field-specific tokenizer
        query_tokens = tokenize_text(query, tokenize_func)
        unique_tokens = np.unique(query_tokens)
        # Retrieve field statistics
        avgdl = stats['avgdl'] # Average Document Length for this field
        dl_dict = stats['DL'] # Dictionary of Document Lengths {doc_id: length}

        for term in unique_tokens:
            # Check if term exists in the index (Document Frequency)
            n_ti = index.df.get(term, 0)
            if n_ti == 0:
                continue

            # Calculate Inverse Document Frequency (IDF)
            idf = math.log(1 + (len(TITLES_DICT) - n_ti + 0.5) / (n_ti + 0.5))

            try:
                # Retrieve the Posting List for the term (list of (doc_id, freq))
                pls = index.read_a_posting_list('.', term, data_bucket_name)
                for doc_id, freq in pls:
                    # Get length of the specific document (default to avgdl if missing)
                    doc_len = dl_dict.get(str(doc_id), avgdl)
                    # Calculate the BM25 denominator (Length Normalization component)
                    denominator = freq + k1 * (1 - b + b * (doc_len / avgdl))
                    # Calculate final BM25 score for this term in this document
                    score = idf * (freq * (k1 + 1)) / denominator
                    # Add weighted score to the candidate's total
                    candidate_scores[doc_id] += (score * weight)
            except Exception:
                continue
    return candidate_scores


def get_semantic_similarity(query, titles):
    """
    Computes semantic similarity between the query and a list of candidate titles
    using a pre-trained Transformer model (e.g., Sentence-BERT).
    """
    if not titles:
        return []
    
    # Encode the query into a dense vector embedding
    query_emb = semantic_model.encode(query, convert_to_tensor=True)
    # Encode all candidate titles into dense vector embeddings
    title_embs = semantic_model.encode(titles, convert_to_tensor=True)
    # Calculate Cosine Similarity between the query vector and title vectors
    semantic_sims = util.cos_sim(query_emb, title_embs)[0].tolist()
    # Returns a list of similarity scores (0.0 to 1.0)
    return semantic_sims


def rank_candidates(query, top_candidates_scores):
    """
    Performs the final re-ranking of candidates using a Hybrid Score.
    Combines Text Match (BM25), Semantic Meaning, Graph Popularity (PageRank), 
    and User Traffic (PageViews).
    """
    # Extract document IDs from the initial BM25 results
    candidate_ids = [int(cid) for cid, _ in top_candidates_scores]

    # Retrieve the actual Title strings for semantic comparison
    titles = [TITLES_DICT.get(str(cid), "Unknown") for cid in candidate_ids]
    # Get semantic similarity scores for these titles
    semantic_sims = get_semantic_similarity(query, titles)

    final_ranked_list = []
    # Iterate through candidates to calculate the composite score
    for i, cid in enumerate(candidate_ids):
        bm25_val = dict(top_candidates_scores).get(cid, 0)
        pr_val = PAGERANK_DICT.get(cid, 0)
        pv_val = math.log1p(WID2PV.get(cid, 0))
        sem_val = semantic_sims[i] if i < len(semantic_sims) else 0

        # Calculate Final Weighted Score (Linear Combination)
        final_score = (bm25_val * 0.5) + (sem_val * 0.3) + (pr_val * 0.1) + (pv_val * 0.1)
        final_ranked_list.append((str(cid), titles[i], final_score))

    # Return the list sorted by final score in descending order
    return sorted(final_ranked_list, key=lambda x: x[2], reverse=True)

In [ ]:
def search(query):
    res = []
    query_tokens = tokenize_smart(query)
    if query_tokens:
        all_bm25_scores = get_bm25_scores(query_tokens)
        if all_bm25_scores:
            top_150 = sorted(all_bm25_scores.items(), key=lambda x: x[1], reverse=True)[:150]
            final_results = rank_candidates(query, top_150)
            res = [(res[0], res[1]) for res in final_results[:100]]
    return res

In [ ]:
#check func
search("im here air force")

[('4101403', 'Hungarian Air Force'),
 ('4045403', 'Foster Air Force Base'),
 ('4085989', 'Congolese Air Force'),
 ('4070196', 'List of United States Air Force Academy alumni'),
 ('4087129', 'Air Force Armament Museum'),
 ('4060580', 'Korat Royal Thai Air Force Base'),
 ('4106232', 'Troy Calhoun'),
 ('4094455', 'Terry W. Virts'),
 ('4094362', 'Gregory H. Johnson'),
 ('4073376', 'Trudy H. Clark'),
 ('4094114', 'Eric Boe'),
 ('4116550', 'RAF Bovingdon'),
 ('4107630', '414th Expeditionary Reconnaissance Squadron'),
 ('4118511', 'Waco Custom Cabin series'),
 ('4072511', 'Weapon systems officer'),
 ('4048119', 'HQ-7'),
 ('4060494', 'RAF East Kirkby'),
 ('4084303', 'James Dutton (astronaut)'),
 ('4054880', 'Hector McGregor'),
 ('4076406', 'Jack Ridley (pilot)'),
 ('4062411', 'RAF Grimsby'),
 ('4120172', 'CAFB'),
 ('4061034', 'Stig Wennerström (colonel)'),
 ('4062612', 'Hugh Saunders'),
 ('4103767', '423d Air Base Group'),
 ('4060466', 'RAF Fiskerton'),
 ('4047145', 'Allahabad Airport'),
 ('41